In [ ]:
# Importation des packages
import pandas as pd
import datetime as dt

# Lecture et merge des DataFrame
df1 = pd.read_csv(r'LFB+Incident+data+from+January+2017.csv', sep = ';')
df2 = pd.read_csv(r'LFB+Mobilisation+data+from+January+2017.csv', sep = ';')
df = df2.merge(df1, on = 'IncidentNumber')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#  Création de la variable cible 'ResponseTimeMinute'
df['DateAndTimeMobilised'] = pd.to_datetime(df['DateAndTimeMobilised'])
df['DateAndTimeArrived'] = pd.to_datetime(df['DateAndTimeArrived'])
df['ResponseTimeMinute'] = (df['DateAndTimeArrived'] - df['DateAndTimeMobilised']).dt.total_seconds()/60

# Création des variables 'HourMobilised', 'WeekdayMobilised', 'MonthMobilised'
df['HourMobilised'] = pd.to_datetime(df['DateAndTimeMobilised']).dt.hour
df['WeekdayMobilised'] = pd.to_datetime(df['DateAndTimeMobilised']).dt.weekday
df['MonthMobilised'] = pd.to_datetime(df['DateAndTimeMobilised']).dt.month

# Création de la variable 'CityCenter'
CityCenter = ['ISLINGTON', 'WESTMINSTER', 'HARINGEY', 'SOUTHWARK', 'TOWER HAMLETS', 'HACKNEY', 'NEWHAM', 'LAMBETH', 'CAMDEN',
               'WANDSWORTH', 'LEWISHAM', 'CITY OF LONDON', 'KENSINGTON AND CHELSEA', 'HAMMERSMITH AND FULHAM']

Peripherie = ['GREENWICH', 'BARKING AND DAGENHAM', 'EALING', 'BRENT', 'REDBRIDGE', 'CROYDON', 'HARROW', 'ENFIELD', 'BROMLEY',
              'BEXLEY', 'HAVERING', 'MERTON', 'HOUNSLOW', 'BARNET', 'HILLINGDON', 'SUTTON', 'WALTHAM FOREST', 
              'KINGSTON UPON THAMES', 'RICHMOND UPON THAMES']

df['CityCenter'] = df['IncGeo_BoroughName'].apply(lambda x: x in CityCenter)

In [ ]:
# Nettoyage de la variable cible

# On cherche les temps de réponse négatifs
# df_new = df[df['ResponseTimeMinute'] < 0] # Il y a 128 incidents concernés (0.02 % des données)

# On cherche les temps de réponse aberrants Ex : ici supérieurs à 1h (60 min)
# df_new = df[df['ResponseTimeMinute'] > 60] # Il y a 679 incidents concernés (0.11 % des données)

# On supprime les incidents pour lesquels le temps de réponse est négatif ou supérieur à 1h (60 min)
df = df[(df['ResponseTimeMinute'] > 0) & (df['ResponseTimeMinute'] < 60)]

In [ ]:
# Sélection des variables d'intérêt

df = df[['IncidentNumber',
         'ResponseTimeMinute',
         'PropertyCategory',
         'AddressQualifier',
         'IncGeo_BoroughName',
         'IncidentStationGround',
         'FirstPumpArriving_DeployedFromStation',
         'DeployedFromStation_Name',
         'DeployedFromLocation',
         'HourMobilised',
         'WeekdayMobilised',
         'MonthMobilised',
         'CityCenter',
         'PumpOrder',
         'CalYear']]

df.head()

,IncidentNumber,ResponseTimeMinute,PropertyCategory,AddressQualifier,IncGeo_BoroughName,IncidentStationGround,FirstPumpArriving_DeployedFromStation,DeployedFromStation_Name,DeployedFromLocation,HourMobilised,WeekdayMobilised,MonthMobilised,CityCenter,PumpOrder,CalYear
0,000004-01012017,14.0,Non Residential,Nearby address - street not listed in gazetteer,GREENWICH,Plumstead,Plumstead,Plumstead,Home Station,0,6,1,False,1,2017
1,000005-01012017,4.0,Dwelling,Correct incident location,ISLINGTON,Euston,Euston,Euston,Home Station,0,6,1,True,1,2017
2,000006-01012017,5.0,Non Residential,Correct incident location,BARKING AND DAGENHAM,Barking,Barking,Barking,Home Station,0,6,1,False,1,2017
3,000008-01012017,7.0,Outdoor,On land associated with building,EALING,Northolt,Northolt,Northolt,Home Station,0,6,1,False,1,2017
4,000008-01012017,7.0,Outdoor,On land associated with building,EALING,Northolt,Northolt,Northolt,Home Station,0,6,1,False,2,2017


In [ ]:
# Recherche et suppression des quelques données manquantes
df.isna().sum()
df = df.dropna(axis = 0, how = 'any')

In [ ]:
# Export du DataFrame nettoyé au format csv
df.to_csv('Pyrefighter_cleaned_database.csv', sep=';', index = False) 